In [6]:
!pip install --upgrade pip
!pip install "snowflake-connector-python[pandas]" "snowflake-snowpark-python[pandas]" snowflake-snowpark-python==1.9.0 fosforio fosforml numpy pandas matplotlib scikit-learn xgboost seaborn python-dateutil tqdm holidays faker
!pip install --upgrade --q snowflake-snowpark-python==1.9.0
!pip uninstall urllib3 -y
!pip install urllib3==1.26.15
!pip install fosforml 

     |████████████████████████████████| 1.8MB 4.0MB/s eta 0:00:01
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 224.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 220.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of fosforml to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of fosforml to determine which version is compatible with other requirements. This could take a while.
  Preparing metadata (setup.py) ... done


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 214.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 237.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 229.7 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 206.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 701.2/701.2 kB 329.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 236.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.9/140.9 kB 286.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 238.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 243.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 192.2 MB/s eta 0:00:00a 0:00:01


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 227.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 222.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 211.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 241.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 254.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 254.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 931.1/931.1 kB 341.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 329.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 323.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 202.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 238.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 MB 247.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mlflow 2.10.0 requires packaging<24, but you have packaging 24.1 which is incompatible.
mlflow 2.10.0 requires pytz<2024, but you have pytz 2024.1 which is incompatible.
dice-ml 0.11 requires pandas<2.0.0, but you have pandas 2.0.0 which is incompatible.
dtale 3.13.1 requires Flask<2.3; python_version >= "3.7", but you have flask 2.3.3 which is incompatible.
dtale 3.13.1 requires matplotlib<=3.7.2; python_version == "3.8", but you have matplotlib 3.7.5 which is incompatible.
fosforml 1.0.0 requires cloudpickle==1.6.0, but you have cloudpickle 2.0.0 which is incompatible.
fosforml 1.0.0 requires PyYAML==6.0, but you have pyyaml 6.0.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 202.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 254.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 261.4 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 171.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 114.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 195.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 192.1 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 333.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 286.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 259.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 269.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 303.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 313.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 208.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 180.2 MB/s eta 0:00:00
   ━━━━━━━━━

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 736.6/736.6 kB 326.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 259.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 335.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 331.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 213.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.3/200.3 MB 254.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 MB 249.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 260.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 348.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 265.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.0/163.0 kB 316.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.7/444.7 kB 334.8 MB/s eta 0:00:00
   ━━━━━━━━━━━

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fosforio 1.0.1 requires pandas==2.0.0, but you have pandas 2.0.3 which is incompatible.
mlflow 2.10.0 requires pyarrow<16,>=4.0.0, but you have pyarrow 16.1.0 which is incompatible.
mlflow 2.10.0 requires pytz<2024, but you have pytz 2024.1 which is incompatible.
refractio 2.1.5.4 requires pandas==2.0.0, but you have pandas 2.0.3 which is incompatible.
dice-ml 0.11 requires pandas<2.0.0, but you have pandas 2.0.3 which is incompatible.
dtale 3.13.1 requires Flask<2.3; python_version >= "3.7", but you have flask 2.3.3 which is incompatible.
dtale 3.13.1 requires matplotlib<=3.7.2; python_version == "3.8", but you have matplotlib 3.7.5 which is incompatible.
mosaic-utils 1.0.2 requires scikit-learn==1.2.1; python_version >= "3.8", but you have scikit-learn 1.3.2 which is incompatible.
refractml 1.0.8 requires cloudp

In [7]:
!pip install dice-ml seaborn imblearn simplejson scipy xgboost pandas dice-ml refractml tabulate numpy scikit-learn pandas-profiling plotly matplotlib scipy statsmodels seaborn dash dtale pydantic-settings

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 224.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 199.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 246.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 642.7/642.7 kB 54.9 MB/s eta 0:00:00


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 287.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of visions to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of visions[type-image-path] to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 165.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 320.4 MB/s eta 0:00:00


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.6/136.6 kB 295.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 263.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 256.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 256.3 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 215.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 236.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 701.2/701.2 kB 335.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 239.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.9/143.9 kB 293.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 318.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 261.2 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 253.9 MB/s eta

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 284.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 338.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 244.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 209.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 MB 154.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 931.1/931.1 kB 326.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 226.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 268.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.1/973.1 kB 285.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 251.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 359.5/359.5 kB 326.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 322.8 MB/s eta 0:00:00
   ━━━━━━━━

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fosforio 1.0.1 requires pandas==2.0.0, but you have pandas 1.5.3 which is incompatible.
mlflow 2.10.0 requires importlib-metadata!=4.7.0,<8,>=3.7.0, but you have importlib-metadata 8.0.0 which is incompatible.
mlflow 2.10.0 requires packaging<24, but you have packaging 24.1 which is incompatible.
mlflow 2.10.0 requires pytz<2024, but you have pytz 2024.1 which is incompatible.
refractio 2.1.5.4 requires pandas==2.0.0, but you have pandas 1.5.3 which is incompatible.
fosforml 1.0.0 requires requests-toolbelt==0.9.1, but you have requests-toolbelt 1.0.0 which is incompatible.
fosforml 1.0.0 requires urllib3==1.26.15, but you have urllib3 1.26.19 which is incompatible.
snowflake-ml-python 1.5.0 requires cloudpickle>=2.0.0, but you have cloudpickle 1.6.0 which is incompatible.
snowflake-ml-python 1.5.0 requires packag

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
from imblearn.over_sampling import SMOTE
from scipy.stats.mstats import winsorize
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import joblib
from fosforml import *
from fosforml.constants import MLModelFlavours
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

Matplotlib created a temporary cache directory at /tmp/matplotlib-stbmdsqr because the default path (/home/mosaic-ai/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
/tmp/pip_packages/_distutils_hack/__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


In [4]:
snowflake.get_connection(connection_name="Final hr attrition data")
df = get_dataframe("Final_HR_Attrition_Data")
df

NameError: name 'get_dataframe' is not defined

In [22]:
df = pd.read_csv("/data/HR_Analytical_Data.csv")
df

,SALARY,SENIORITY,TENURE MONTHS,MONTHS AFTER COLLEGE,BIRTH YEAR,AGE,MAPPED ROLE CLEAN,SEX,ETHNICITY,HOSPITAL TYPE,...,OVER TIME HOURS,CHURN,BUSINESS TRAVEL,ENVIRONMENT SATISFACTION,JOB SATISFACTION,MARTIAL STATUS,PERCENTAGE SALARY HIKE,PERFORMANCE RATING,RELATIONSHIP SATISFACTION,WORK LIFE BALANCE
0,68271.546,3,8,116,1984,40,occupational,F,Black,Childrens,...,15,False,Travel_Rarely,High,Low,Married,11,Excellent,Medium,Best
1,70471.235,3,37,18,1992,32,nurse,F,Hispanic,Psychiatric,...,10,True,Travel_Rarely,High,Medium,Married,11,Outstanding,Low,Good
2,31657.073,1,13,161,1986,38,occupational,F,Black,Psychiatric,...,7,True,Travel_Rarely,High,Medium,Married,18,Excellent,Medium,Better
3,26898.652,1,57,3,1981,43,technologist,M,API,Acute Care Hospitals,...,14,True,Travel_Rarely,Very High,Medium,Divorced,12,Low,Very High,Good
4,53490.238,2,47,16,1988,36,emt,F,White,Psychiatric,...,14,False,Travel_Rarely,Very High,Low,Single,19,Good,Low,Bad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,79047.028,5,106,164,1975,49,technologist,M,Hispanic,Acute Care - Department of Defense,...,6,True,Travel_Rarely,Low,Medium,Divorced,14,Excellent,Medium,Best
299996,51505.375,1,12,69,1985,39,social,F,Multiple,Acute Care - Department of Defense,...,9,False,Travel_Rarely,Low,Medium,Married,18,Good,Very High,Best
299997,97520.177,5,45,113,1969,55,physician,M,Hispanic,Acute Care Hospitals,...,1,False,Travel_Rarely,High,High,Divorced,20,Outstanding,Low,Best
299998,55666.168,2,32,5,1987,37,emt,M,White,Childrens,...,9,True,Travel_Rarely,Medium,Low,Married,23,Low,Very High,Best


In [23]:
print(df.isnull().sum())

SALARY                           0
SENIORITY                        0
TENURE MONTHS                    0
MONTHS AFTER COLLEGE             0
BIRTH YEAR                       0
AGE                              0
MAPPED ROLE CLEAN                0
SEX                              0
ETHNICITY                        0
HOSPITAL TYPE                    0
HOSPITAL OWNERSHIP               0
COMPANY NAME                     0
CITY                             0
STATE                            0
DISTANCE                         0
DEGREE CLEAN                     0
SCHOOL END DATE                  0
JOB START DATE                   0
JOB END DATE                 90102
USER ID                          0
OVER TIME HOURS                  0
CHURN                            0
BUSINESS TRAVEL                  0
ENVIRONMENT SATISFACTION         0
JOB SATISFACTION                 0
MARTIAL STATUS                   0
PERCENTAGE SALARY HIKE           0
PERFORMANCE RATING               0
RELATIONSHIP SATISFA

In [24]:
df = df.dropna()

In [25]:
print(df.isnull().sum())

SALARY                       0
SENIORITY                    0
TENURE MONTHS                0
MONTHS AFTER COLLEGE         0
BIRTH YEAR                   0
AGE                          0
MAPPED ROLE CLEAN            0
SEX                          0
ETHNICITY                    0
HOSPITAL TYPE                0
HOSPITAL OWNERSHIP           0
COMPANY NAME                 0
CITY                         0
STATE                        0
DISTANCE                     0
DEGREE CLEAN                 0
SCHOOL END DATE              0
JOB START DATE               0
JOB END DATE                 0
USER ID                      0
OVER TIME HOURS              0
CHURN                        0
BUSINESS TRAVEL              0
ENVIRONMENT SATISFACTION     0
JOB SATISFACTION             0
MARTIAL STATUS               0
PERCENTAGE SALARY HIKE       0
PERFORMANCE RATING           0
RELATIONSHIP SATISFACTION    0
WORK LIFE BALANCE            0
dtype: int64


In [26]:
df = df.drop(["USER ID", "JOB START DATE", "JOB END DATE", "SCHOOL END DATE"], axis = 1)

In [27]:
from sklearn.model_selection import train_test_split

In [28]:
CATEGORICAL_COLUMNS = ["MAPPED ROLE CLEAN","SEX", "ETHNICITY","HOSPITAL TYPE", "HOSPITAL OWNERSHIP","COMPANY NAME","CITY","STATE","DISTANCE", 
                       "DEGREE CLEAN","BUSINESS TRAVEL","ENVIRONMENT SATISFACTION","JOB SATISFACTION","MARTIAL STATUS","PERFORMANCE RATING","RELATIONSHIP SATISFACTION","WORK LIFE BALANCE"]
NUMERICAL_COLUMNS = ["SALARY", "SENIORITY", "TENURE MONTHS", "MONTHS AFTER COLLEGE", "BIRTH YEAR","AGE", "OVER TIME HOURS", "PERCENTAGE SALARY HIKE",]
LABEL_COLUMNS = ["CHURN"]
OUTPUT_COLUMNS = ["PREDICTION"]

In [29]:
feature_columns = CATEGORICAL_COLUMNS + NUMERICAL_COLUMNS 

In [30]:
feature_columns = [col for col in feature_columns if col in df.columns]
LABEL_COLUMNS = [col for col in LABEL_COLUMNS if col in df.columns]

In [31]:
X = df[feature_columns]
Y = df[LABEL_COLUMNS]

In [32]:
X.head()

,MAPPED ROLE CLEAN,SEX,ETHNICITY,HOSPITAL TYPE,HOSPITAL OWNERSHIP,COMPANY NAME,CITY,STATE,DISTANCE,DEGREE CLEAN,...,RELATIONSHIP SATISFACTION,WORK LIFE BALANCE,SALARY,SENIORITY,TENURE MONTHS,MONTHS AFTER COLLEGE,BIRTH YEAR,AGE,OVER TIME HOURS,PERCENTAGE SALARY HIKE
0,occupational,F,Black,Childrens,Department of Defense,Village Practice Management Co. LLC,Lewiston,ME,>10 Miles,Professional Certificate,...,Medium,Best,68271.546,3,8,116,1984,40,15,11
1,nurse,F,Hispanic,Psychiatric,Proprietary,McKesson Corp.,Charlottesville,VA,>10 Miles,Undergraduate Diploma,...,Low,Good,70471.235,3,37,18,1992,32,10,11
2,occupational,F,Black,Psychiatric,Government - Local,24-7 Bright Star Healthcare LLC,Jacksonville,FL,5-10 Miles,Bachelors Degree,...,Medium,Better,31657.073,1,13,161,1986,38,7,18
4,emt,F,White,Psychiatric,Government - Hospital District or Authority,24-7 Bright Star Healthcare LLC,Evansville,IN,2-5 Miles,Masters Degree,...,Low,Bad,53490.238,2,47,16,1988,36,14,19
5,nurse,M,Hispanic,Acute Care Hospitals,Government - Hospital District or Authority,AmerisourceBergen Corp.,Lakeland,FL,<2 Miles,Undergraduate Diploma,...,Medium,Bad,47749.212,1,40,32,1983,41,3,13


In [33]:
Y.head()

,CHURN
0,False
1,True
2,True
4,False
5,False


In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size = 0.20, random_state = 42)

In [35]:
## train_df and test_df are both snowpark dataframes
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline

categorical_transformer = make_pipeline(
    SimpleImputer(strategy='constant', fill_value='missing'),
    OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
)

numerical_transformer = make_pipeline(
    SimpleImputer(strategy='mean'),
    MinMaxScaler(clip=True)
)

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, CATEGORICAL_COLUMNS),
        ('num', numerical_transformer, NUMERICAL_COLUMNS)
    ]
)


pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier())
])

pipeline.fit(X_train, y_train)

result = pipeline.predict(X_test)

In [36]:
result

array([1, 0, 1, ..., 0, 1, 1])

In [37]:
from joblib import dump, load
filename = "HR_Attrition_ml_model.joblib"
dump(pipeline, filename)

['HR_Attrition_ml_model.joblib']

In [38]:
import requests

In [ ]:
test = X_test
train = X_train
train["CHURN"] = train["CHURN"].apply(lambda x: 1 if x else 0)
pred = model.predict(test)
pred["probabilities"] = model.predict_proba(test)[["predict_proba_True"]]
test["PREDICTION"] = pred["PREDICTION"]
test["probabilities"] = pred["probabilities"]
test["CHURN"] = test["CHURN"].apply(lambda x: 1 if x else 0)

y_prob = np.array(test["probabilities"])
y_test = pred["CHURN"]
y_pred = pred["PREDICTION"]
X_train = train.drop(["CHURN"], axis=1)
X_test = pred.drop(["CHURN", "PREDICTION", "probabilities"], axis=1)
y_train = train["CHURN"]

In [39]:
y_pred = pipeline.predict (X_test)
y_prob = pipeline.predict_proba(X_test)

In [40]:
#@scoring_func
def score(model, request):
    payload_dict = request.json["payload"]
    data = pd.DataFrame(payload_dict,index=[0])
    prediction = str(model.predict(data)[0])
    return prediction

In [42]:
## registering the model in refract.
tmp = register_model(pipeline, 
               score, 
               name="HR_Analytic_ml_model_Final", 
               description="Final_HR_ANALYTICS_model_trained_using _ml",
               flavour=MLModelFlavours.sklearn,
               model_type="classification",
               init_script="\\n pip install fosforml \\n pip install fosforio[snowflake] \\n pip install seaborn \\n pip install snowflake-connector-python[pandas] \\n pip install joblib==1.3.2 scikit-learn=1.3.2",
               y_true=y_test,
               y_pred=y_pred, 
               features=X_train.columns,
               labels=[0,1],
               input_type="json", 
               explain_ai=True,
               prob=y_prob,
               x_train=X_train, 
               x_test=X_test, 
               y_train=y_train,
               y_test=y_test,
               feature_names=X_train.columns.tolist(),
               original_features=X_train.columns.tolist(),
               feature_ids=X_train.columns,
               target_names=['NOT LEFT','LEFT'],
               kyd=True, kyd_score = True)


Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%


In [43]:
payload  = {"payload": X_test.iloc[0].to_dict()}
payload

{'payload': {'MAPPED ROLE CLEAN': 'social',
  'SEX': 'M',
  'ETHNICITY': 'White',
  'HOSPITAL TYPE': 'Psychiatric',
  'HOSPITAL OWNERSHIP': 'Government - State',
  'COMPANY NAME': 'Encompass Health Corp.',
  'CITY': 'York',
  'STATE': ' PA',
  'DISTANCE': '<2 Miles',
  'DEGREE CLEAN': 'Undergraduate Diploma',
  'BUSINESS TRAVEL': 'Non-Travel',
  'ENVIRONMENT SATISFACTION': 'High',
  'JOB SATISFACTION': 'Very High',
  'MARTIAL STATUS': 'Married',
  'PERFORMANCE RATING': 'Low',
  'RELATIONSHIP SATISFACTION': 'High',
  'WORK LIFE BALANCE': 'Better',
  'SALARY': 60154.454,
  'SENIORITY': 1,
  'TENURE MONTHS': 32,
  'MONTHS AFTER COLLEGE': 19,
  'BIRTH YEAR': 1990,
  'AGE': 34,
  'OVER TIME HOURS': 6,
  'PERCENTAGE SALARY HIKE': 18}}